### HOA Network Selection for...
##Efficiently Identifying Task Groupings for Multi-Task Learning

Licensed under the Apache License, Version 2.0


### CelebA

In [ ]:
# HOA Groupings on the Validation Set.
revised_integrals = {'5_o_Clock_Shadow': {'5_o_Clock_Shadow': 92.28, 'Black_Hair': 91.73, 'Blond_Hair': 92.09, 'Brown_Hair': 92.16, 'Goatee': 92.34, 
					   'Mustache': 92.30, 'No_Beard': 92.18, 'Rosy_Cheeks': 92.12, 'Wearing_Hat': 92.20}, 
					 'Black_Hair': {'5_o_Clock_Shadow': 90.41, 'Black_Hair': 90.19, 'Blond_Hair': 90.29, 'Brown_Hair': 90.48, 'Goatee': 90.24, 
					   'Mustache': 90.46, 'No_Beard': 90.30, 'Rosy_Cheeks': 90.22, 'Wearing_Hat': 90.34}, 
					 'Blond_Hair': {'5_o_Clock_Shadow': 94.87, 'Black_Hair': 95.00, 'Blond_Hair': 94.96, 'Brown_Hair': 94.87, 'Goatee': 94.95, 
					   'Mustache': 94.94, 'No_Beard': 94.94, 'Rosy_Cheeks': 94.75, 'Wearing_Hat': 94.98}, 
					 'Brown_Hair': {'5_o_Clock_Shadow': 84.44, 'Black_Hair': 84.42, 'Blond_Hair': 84.39, 'Brown_Hair': 84.39, 'Goatee': 84.09, 
					   'Mustache': 84.31, 'No_Beard': 84.30, 'Rosy_Cheeks': 84.39, 'Wearing_Hat': 84.39}, 
					 'Goatee': {'5_o_Clock_Shadow': 95.84, 'Black_Hair': 95.76, 'Blond_Hair': 95.81, 'Brown_Hair': 95.77, 'Goatee': 95.90, 
					   'Mustache': 95.87, 'No_Beard': 96.07, 'Rosy_Cheeks': 95.83, 'Wearing_Hat': 95.83}, 
					 'Mustache': {'5_o_Clock_Shadow': 95.69, 'Black_Hair': 95.68, 'Blond_Hair': 95.80, 'Brown_Hair': 95.77, 'Goatee': 95.75, 
					   'Mustache': 95.84, 'No_Beard': 95.75, 'Rosy_Cheeks': 95.82, 'Wearing_Hat': 95.74}, 
					 'No_Beard': {'5_o_Clock_Shadow': 94.14, 'Black_Hair': 94.01, 'Blond_Hair': 93.99, 'Brown_Hair': 94.17, 'Goatee': 94.47, 
					   'Mustache': 94.30, 'No_Beard': 94.44, 'Rosy_Cheeks': 93.96, 'Wearing_Hat': 94.14}, 
					 'Rosy_Cheeks': {'5_o_Clock_Shadow': 94.66, 'Black_Hair': 94.69, 'Blond_Hair': 94.64, 'Brown_Hair': 94.72, 'Goatee': 94.68, 
					   'Mustache': 94.62, 'No_Beard': 94.62, 'Rosy_Cheeks': 94.64, 'Wearing_Hat': 94.59}, 
					 'Wearing_Hat': {'5_o_Clock_Shadow': 98.61, 'Black_Hair': 98.63, 'Blond_Hair': 98.64, 'Brown_Hair': 98.56, 'Goatee': 98.69, 
					   'Mustache': 98.66, 'No_Beard': 98.67, 'Rosy_Cheeks': 98.59, 'Wearing_Hat': 98.69}}

# Minimize Error rather than Maximize Accuracy
for key in revised_integrals:
	for task in revised_integrals[key]:
		revised_integrals[key][task] = 100.0 - revised_integrals[key][task]

### Taskonomy

In [ ]:
# HOA Groupings on the Validation Set.
revised_integrals = {'s': {'s': 0.0509, 'd': 0.0528, 'n': 0.0471, 't': 0.0606, 'k': 0.0655},
                     'd': {'s': 0.2636, 'd': 0.2616, 'n': 0.2551, 't': 0.2675, 'k': 0.2674},
                     'n': {'s': 0.1004, 'd': 0.1002, 'n': 0.0975, 't': 0.1082, 'k': 0.1110},
                     't': {'s': 0.0287, 'd': 0.0423, 'n': 0.0310, 't': 0.0337, 'k': 0.0232},
                     'k': {'s': 0.0972, 'd': 0.1068, 'n': 0.0568, 't': 0.0933, 'k': 0.0910}}

### Network Selection Algorithm

In [ ]:
import math 
import numpy as np

def gen_task_combinations(tasks, rtn, index, path, path_dict):
  if index >= len(tasks):
    return 

  for i in range(index, len(tasks)):
    cur_task = tasks[i]
    new_path = path
    new_dict = {k:v for k,v in path_dict.items()}
    
    # Building from a tree with two or more tasks...
    if new_path:
      new_dict[cur_task] = 0.
      for prev_task in path_dict:
        new_dict[prev_task] += revised_integrals[prev_task][cur_task]
        new_dict[cur_task] += revised_integrals[cur_task][prev_task]
      new_path = '{}|{}'.format(new_path, cur_task)
      rtn[new_path] = new_dict
    else: # First element in a new-formed tree
      new_dict[cur_task] = 0.
      new_path = cur_task

    gen_task_combinations(tasks, rtn, i+1, new_path, new_dict)

    # Fix single-task accuracy since dfs is finished for this task.
    if '|' not in new_path:
      new_dict[cur_task] = revised_integrals[cur_task][cur_task]
      rtn[new_path] = new_dict

In [ ]:
rtn = {}
tasks = list(revised_integrals.keys())
num_tasks = len(tasks)
task_combinations = gen_task_combinations(tasks=tasks, rtn=rtn, index=0, path='', path_dict={})

# Normalize by the number of times the accuracy of any given element has been summed. 
# i.e. (a,b,c) => [acc(a|b) + acc(a|c)]/2 + [acc(b|a) + acc(b|c)]/2 + [acc(c|a) + acc(c|b)]/2
for group in rtn:
  if '|' in group:
    for task in rtn[group]:
      rtn[group][task] /= (len(group.split('|')) - 1)

print(rtn)
assert(len(rtn.keys()) == 2**len(revised_integrals.keys()) - 1)
rtn_tup = [(key,val) for key,val in rtn.items()]

In [ ]:
def select_groups(index, cur_group, best_group, best_val, splits):
  # Check if this group covers all tasks.
  task_set = set()
  for group in cur_group:
    for task in group.split('|'): task_set.add(task)
  if len(task_set) == num_tasks:
    best_tasks = {task:1e6 for task in task_set}
    
    # Compute the per-task best scores for each task and average them together.
    for group in cur_group:
      for task in cur_group[group]:
        # Minimize error.
        best_tasks[task] = min(best_tasks[task], cur_group[group][task])
    group_avg = np.mean(list(best_tasks.values()))
    
    # Compare with the best grouping seen thus far.
    if group_avg < best_val[0]:
      print(cur_group)
      best_val[0] = group_avg
      best_group.clear()
      for entry in cur_group:
        best_group[entry] = cur_group[entry]
  
  # Base case.
  if len(cur_group.keys()) == splits:
    return

  # Back to combinatorics 
  for i in range(index, len(rtn_tup)):
    selected_group, selected_dict = rtn_tup[i]

    new_group = {k:v for k,v in cur_group.items()}
    new_group[selected_group] = selected_dict

    if len(new_group.keys()) <= splits:
      select_groups(i + 1, new_group, best_group, best_val, splits)

selected_group = {}
selected_val = [100000000]
select_groups(index=0, cur_group={}, best_group=selected_group, best_val=selected_val, splits=2)
print(selected_group)
print(selected_val)